In [ ]:
from pydub import AudioSegment
import os
import string

def generate_unique_filename(base_name, extension, output_dir):
    """
    Generate a unique filename by adding letters to the base name if the file already exists.
    """
    i = 0
    filename = f"{base_name}{extension}"
    
    while os.path.exists(os.path.join(output_dir, filename)):
        i += 1
        suffix = ""
        temp = i
        while temp > 0:
            temp -= 1
            suffix = string.ascii_lowercase[temp % len(string.ascii_lowercase)] + suffix
            temp //= len(string.ascii_lowercase)
        filename = f"{base_name}{suffix}{extension}"

    return filename

def mix_audio_files(audio1, audio2, noise_weight, sample_rate, duration1, duration2):
    """
    Mixes two audio files with given parameters.
    """
    # Set frame rate for both audio files
    audio1 = audio1.set_frame_rate(sample_rate)
    audio2 = audio2.set_frame_rate(sample_rate)
    
    # Trim audio files to specified durations if they are longer
    if len(audio1) > duration1:
        audio1 = audio1[:duration1]
    if len(audio2) > duration2:
        audio2 = audio2[:duration2]
    
    # Apply gain to audio2
    audio2 = audio2.apply_gain(20 * (noise_weight - 1))
    
    # Determine the longer and shorter audio files
    if len(audio1) > len(audio2):
        longer_audio = audio1
        shorter_audio = audio2
    else:
        longer_audio = audio2
        shorter_audio = audio1
    
    # Calculate the starting position to place a short audio file in the middle of a long
    start_position = (len(longer_audio) - len(shorter_audio)) // 2
    
    # Mix the audio
    combined = longer_audio.overlay(shorter_audio, position=start_position)
    
    return combined

def mix_all_audios(dir1, dir2, noise_weights, sample_rate, output_dir):
    # List files in dir1 and dir2
    files1 = [f for f in os.listdir(dir1) if f.endswith('.wav')]
    files2 = [f for f in os.listdir(dir2) if f.endswith('.wav')]
    
    # Create output dir if not created
    os.makedirs(output_dir, exist_ok=True)
    
    # Define durations in milliseconds
    duration1 = 5 * 1000  # 5 seconds
    duration2 = 10 * 1000  # 10 seconds
    
    # Step 1: Mix files from dir1 and dir2 and save them
    intermediate_files = []
    for file1 in files1:
        for file2 in files2:
            for noise_weight in noise_weights:
                file1_path = os.path.join(dir1, file1)
                file2_path = os.path.join(dir2, file2)
                
                try:
                    # Load audio files
                    audio1 = AudioSegment.from_file(file1_path)
                    audio2 = AudioSegment.from_file(file2_path)
                except Exception as e:
                    print(f"Error loading {file1} and {file2}: {e}")
                    continue
                
                # Mix the audio files
                combined = mix_audio_files(audio1, audio2, noise_weight, sample_rate, duration1, duration2)
                
                # Format the name of the new file and ensure it's unique
                base_filename = os.path.splitext(file1)[0]
                output_filename = generate_unique_filename(base_filename, ".wav", output_dir)
                output_path = os.path.join(output_dir, output_filename)
                
                # Save the new file
                combined.export(output_path, format="wav")
                
                # Store the path of the intermediate file
                intermediate_files.append(output_path)
                
                print(f"Mixed audio file saved to: {output_path}")
    
    # Step 2: Re-mix the generated files with the files from dir2
    for interm_file in intermediate_files:
        for file2 in files2:
            for noise_weight in noise_weights:
                interm_audio = AudioSegment.from_file(interm_file)
                file2_path = os.path.join(dir2, file2)
                
                try:
                    # Load the second audio file
                    audio2 = AudioSegment.from_file(file2_path)
                except Exception as e:
                    print(f"Error loading {file2} for re-mixing with {interm_file}: {e}")
                    continue
                
                # Mix the audio files again
                combined = mix_audio_files(interm_audio, audio2, noise_weight, sample_rate, duration1, duration2)
                
                # Format the name of the new file and ensure it's unique
                base_filename = os.path.splitext(os.path.basename(interm_file))[0]
                output_filename = generate_unique_filename(base_filename, ".wav", output_dir)
                output_path = os.path.join(output_dir, output_filename)
                
                # Save the new file
                combined.export(output_path, format="wav")
                
                print(f"Re-mixed audio file saved to: {output_path}")

# Example of usage
dir1 = r"F:\test\normal_signal"
dir2 = r"F:\test\Background"
noise_weights = [0.5, 1.0]  # List of noise weights
sample_rate = 44100
output_dir = r"F:\test\filestrain"

# Call the function to mix all audio files
mix_all_audios(dir1, dir2, noise_weights, sample_rate, output_dir)